In [1]:
%matplotlib inline
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os
import scipy.io as sio
import matplotlib
import logging

import utils as utils
from matplotlib import cm

In [2]:
runname = 'debug_withC_rogers_netcdfBdy1_EB_soliton_f_withsponge_focus_finey_summer'
datafilepath = '/Volumes/Tan_2023/projects_backup/SUNTANS/HPC/'+runname+'/data/'

# datafilename = 'sun_out.nc_0000.nc'
datafilename = 'sun_sparse.nc_0000.nc'

# path to saved result
resultfilepath = datafilepath[:-5] + 'mfiles/results/'
# path to saved figures
outfilepath = '../figure/'
if not os.path.exists(resultfilepath):
    os.mkdir(resultfilepath)
if not os.path.exists(outfilepath):
    os.mkdir(outfilepath)

param_data = {
    "datafilepath": datafilepath,
    "netcdf_filename": datafilename,
            }
param_result = {
    "resultfilepath": resultfilepath,
    "result_filename": 'sun_out_energy_derived.nc',
            }
idx_end = -15

# Organize data output

In [3]:
# read data
Data = xr.open_dataset(datafilepath+datafilename)

# grid info
import mat73
grid = mat73.loadmat('/Volumes/Tan_2023/projects_backup/SUNTANS/HPC/'+runname+'/mfiles/SUNTANS_grid.mat', only_include=['Nk', 'Nx', 'Ny', 'Depth']) 

Nx = grid['Nx'].astype('int')
Ny = grid['Ny'].astype('int')
Nk = grid['Nk'].astype('int')
Nc = Nx*Ny
# Depth = np.reshape(grid['Depth'].astype('int'), (Nx, Ny), order='F')
Depth = np.reshape(grid['Depth'], (Nx, Ny), order='F')

xv = np.reshape(Data.xv.data, (Nx, Ny), order='F')
yv = np.reshape(Data.yv.data, (Nx, Ny), order='F')

## _derived

In [4]:
dEk_prime = np.reshape(Data.dEk_prime.data, (len(Data.time), Nx, Ny), order='F')
dEp_prime = np.reshape(Data.dEp_prime.data, (len(Data.time), Nx, Ny), order='F')

In [5]:
dEp0 = np.reshape(Data.dEp0.data, (len(Data.time), Nx, Ny), order='F')
dEk0 = np.reshape(Data.dEk0.data, (len(Data.time), Nx, Ny), order='F')

data = xr.Dataset({'dEp0': (['time','x', 'y'], dEp0),
                   'dEp_prime': (['time','x', 'y'], dEp_prime),
                   'dEk0': (['time','x', 'y'], dEk0),
                   'dEk_prime': (['time','x', 'y'], dEk_prime),
                  },
                  coords={'x': xv[:,0],
                          'y': yv[0,:],
                          'z': -Data.z_r.data,
                          'time': Data.time.data},
                  attrs={'title': 'SUNTANS internally calculated energy budget'}) 

del dEp0, dEp_prime, dEk0, dEk_prime

C1 = np.reshape(Data.C1_int.data, (len(Data.time), Nx, Ny), order='F')
C2 = np.reshape(Data.C2_int.data, (len(Data.time), Nx, Ny), order='F')
data["C_int"]=(['time', 'x', 'y'],  C1+C2)

del C1, C2

D_0 = np.reshape(Data.D_0_int.data, (len(Data.time), Nx, Ny), order='F')
D_prime = np.reshape(Data.D_prime_int.data, (len(Data.time), Nx, Ny), order='F')
data["D0"]=(['time', 'x', 'y'],  D_0)
data["D_prime"]=(['time', 'x', 'y'],  D_prime)

del D_0, D_prime

data["depth"]=(['x', 'y'],  Depth)

data.to_netcdf(param_result['resultfilepath']+param_result['result_filename'])

In [6]:
del data

## _bt_flux

In [4]:
Fx_01 = np.reshape(Data.Fx_01_int.data, (len(Data.time), Nx, Ny), order='F')
Fx_02 = np.reshape(Data.Fx_02_int.data, (len(Data.time), Nx, Ny), order='F')
Fx_03 = np.reshape(Data.Fx_03_int.data, (len(Data.time), Nx, Ny), order='F')
Fx_04 = np.reshape(Data.Fx_04_int.data, (len(Data.time), Nx, Ny), order='F')

data_bt = xr.Dataset({'Fx_01_int': (['time','x', 'y'], Fx_01),
                  },
                  coords={'x': xv[:,0],
                          'y': yv[0,:],
                          'z': -Data.z_r.data,
                          'time': Data.time.data},
                  attrs={'title': 'SUNTANS internally calculated energy budget'}) 
data_bt["Fx_02_int"]=(['time', 'x', 'y'],  Fx_02)
data_bt["Fx_03_int"]=(['time', 'x', 'y'],  Fx_03)
data_bt["Fx_04_int"]=(['time', 'x', 'y'],  Fx_04)

del Fx_01, Fx_02, Fx_03, Fx_04

Fy_01 = np.reshape(Data.Fy_01_int.data, (len(Data.time), Nx, Ny), order='F')
Fy_02 = np.reshape(Data.Fy_02_int.data, (len(Data.time), Nx, Ny), order='F')
Fy_03 = np.reshape(Data.Fy_03_int.data, (len(Data.time), Nx, Ny), order='F')
Fy_04 = np.reshape(Data.Fy_04_int.data, (len(Data.time), Nx, Ny), order='F')

data_bt["Fy_01_int"]=(['time', 'x', 'y'],  Fy_01)
data_bt["Fy_02_int"]=(['time', 'x', 'y'],  Fy_02)
data_bt["Fy_03_int"]=(['time', 'x', 'y'],  Fy_03)
data_bt["Fy_04_int"]=(['time', 'x', 'y'],  Fy_04)

del Fy_01, Fy_02, Fy_03, Fy_04

data_bt.to_netcdf(param_result['resultfilepath']+param_result['result_filename'][:-3]+'_bt_flux.nc')

In [5]:
del data_bt

## _bc_flux

In [4]:
Fx_prime1 = np.reshape(Data.Fx_prime1_int.data, (len(Data.time), Nx, Ny), order='F')
Fx_prime2 = np.reshape(Data.Fx_prime2_int.data, (len(Data.time), Nx, Ny), order='F')
Fx_prime3 = np.reshape(Data.Fx_prime3_int.data, (len(Data.time), Nx, Ny), order='F')
Fx_prime4 = np.reshape(Data.Fx_prime4_int.data, (len(Data.time), Nx, Ny), order='F')
Fx_prime5 = np.reshape(Data.Fx_prime5_int.data, (len(Data.time), Nx, Ny), order='F')

data_bc = xr.Dataset({'Fx_prime1_int': (['time','x', 'y'], Fx_prime1),
                  },
                  coords={'x': xv[:,0],
                          'y': yv[0,:],
                          'z': -Data.z_r.data,
                          'time': Data.time.data},
                  attrs={'title': 'SUNTANS internally calculated energy budget'}) 
data_bc["Fx_prime2_int"]=(['time', 'x', 'y'],  Fx_prime2)
data_bc["Fx_prime3_int"]=(['time', 'x', 'y'],  Fx_prime3)
data_bc["Fx_prime4_int"]=(['time', 'x', 'y'],  Fx_prime4)
data_bc["Fx_prime5_int"]=(['time', 'x', 'y'],  Fx_prime5)

del Fx_prime1, Fx_prime2, Fx_prime3, Fx_prime4, Fx_prime5

Fy_prime1 = np.reshape(Data.Fy_prime1_int.data, (len(Data.time), Nx, Ny), order='F')
Fy_prime2 = np.reshape(Data.Fy_prime2_int.data, (len(Data.time), Nx, Ny), order='F')
Fy_prime3 = np.reshape(Data.Fy_prime3_int.data, (len(Data.time), Nx, Ny), order='F')
Fy_prime4 = np.reshape(Data.Fy_prime4_int.data, (len(Data.time), Nx, Ny), order='F')
Fy_prime5 = np.reshape(Data.Fy_prime5_int.data, (len(Data.time), Nx, Ny), order='F')

data_bc["Fy_prime1_int"]=(['time', 'x', 'y'],  Fy_prime1)
data_bc["Fy_prime2_int"]=(['time', 'x', 'y'],  Fy_prime2)
data_bc["Fy_prime3_int"]=(['time', 'x', 'y'],  Fy_prime3)
data_bc["Fy_prime4_int"]=(['time', 'x', 'y'],  Fy_prime4)
data_bc["Fy_prime5_int"]=(['time', 'x', 'y'],  Fy_prime5)

del Fy_prime1, Fy_prime2, Fy_prime3, Fy_prime4, Fy_prime5
del Data

data_bc.to_netcdf(param_result['resultfilepath']+param_result['result_filename'][:-3]+'_bc_flux.nc')

In [6]:
del data_bc

## _derived

In [8]:
data_bt_flux = xr.open_dataset(param_result['resultfilepath']+param_result['result_filename'][:-3]+'_bt_flux.nc')

div_F01 = data_bt_flux.Fx_01_int.differentiate("x") + data_bt_flux.Fy_01_int.differentiate("y")
div_F02 = data_bt_flux.Fx_02_int.differentiate("x") + data_bt_flux.Fy_02_int.differentiate("y")
div_F03 = data_bt_flux.Fx_03_int.differentiate("x") + data_bt_flux.Fy_03_int.differentiate("y")
div_F04 = data_bt_flux.Fx_04_int.differentiate("x") + data_bt_flux.Fy_04_int.differentiate("y")

del data_bt_flux

In [13]:
data_bt = xr.Dataset({'div_F01': (['time','x', 'y'], div_F01.data),
                  },
                  coords={'x': xv[:,0],
                          'y': yv[0,:],
                          'z': -Data.z_r.data,
                          'time': Data.time.data},
                  attrs={'title': 'SUNTANS internally calculated energy flux divergence'}) 

data_bt["div_F02"]=(['time', 'x', 'y'],  div_F02.data)
data_bt["div_F03"]=(['time', 'x', 'y'],  div_F03.data)
data_bt["div_F04"]=(['time', 'x', 'y'],  div_F04.data)

data_bt.to_netcdf(param_result['resultfilepath']+param_result['result_filename'][:-3]+'_bt.nc')

del div_F01, div_F02, div_F03, div_F04, data_bt

In [4]:
# read data
data_bc_flux = xr.open_dataset(param_result['resultfilepath']+param_result['result_filename'][:-3]+'_bc_flux.nc')

div_Fprime1 = data_bc_flux.Fx_prime1_int.differentiate("x") + data_bc_flux.Fy_prime1_int.differentiate("y")
div_Fprime2 = data_bc_flux.Fx_prime2_int.differentiate("x") + data_bc_flux.Fy_prime2_int.differentiate("y")
div_Fprime3 = data_bc_flux.Fx_prime3_int.differentiate("x") + data_bc_flux.Fy_prime3_int.differentiate("y")
div_Fprime4 = data_bc_flux.Fx_prime4_int.differentiate("x") + data_bc_flux.Fy_prime4_int.differentiate("y")
div_Fprime5 = data_bc_flux.Fx_prime5_int.differentiate("x") + data_bc_flux.Fy_prime5_int.differentiate("y")

del data_bc_flux

In [6]:
data_bc = xr.Dataset({'div_Fprime1': (['time','x', 'y'], div_Fprime1.data),
                  },
                  coords={'x': xv[:,0],
                          'y': yv[0,:],
                          'z': -Data.z_r.data,
                          'time': Data.time.data},
                  attrs={'title': 'SUNTANS internally calculated energy flux divergence'}) 

data_bc["div_Fprime2"]=(['time', 'x', 'y'],  div_Fprime2.data)
data_bc["div_Fprime3"]=(['time', 'x', 'y'],  div_Fprime3.data)
data_bc["div_Fprime4"]=(['time', 'x', 'y'],  div_Fprime4.data)
data_bc["div_Fprime5"]=(['time', 'x', 'y'],  div_Fprime5.data)

data_bc.to_netcdf(param_result['resultfilepath']+param_result['result_filename'][:-3]+'_bc.nc')

del div_Fprime1, div_Fprime2, div_Fprime3, div_Fprime4, div_Fprime5, data_bc

In [14]:
data_bt = xr.open_dataset(param_result['resultfilepath']+param_result['result_filename'][:-3]+'_bt.nc')
data_bc = xr.open_dataset(param_result['resultfilepath']+param_result['result_filename'][:-3]+'_bc.nc')
data = xr.open_dataset(param_result['resultfilepath']+param_result['result_filename'])

dissip0 = -data.dEp0 - data.dEk0 - data_bt.div_F01 - data_bt.div_F02 - data_bt.div_F03 - data_bt.div_F04 - data.C_int - data.D0
dissip_prime = -data.dEp_prime - data.dEk_prime - data_bc.div_Fprime1 - data_bc.div_Fprime2 - data_bc.div_Fprime3 - data_bc.div_Fprime4 - data_bc.div_Fprime5 + data.C_int - data.D_prime

data["dissip0"]=(['time', 'x', 'y'],  dissip0.data)
data["dissip_prime"]=(['time', 'x', 'y'],  dissip_prime.data)

In [19]:
data.to_netcdf(param_result['resultfilepath']+param_result['result_filename'])

del data_bt, data_bc, data